In [1]:
import random
import math

# invert a dictionary of lists (assuming no duplicates)
def invert_dictlist(d):
    return dict( (v,k) for k in d for v in d[k] )

# invert a dictionary of lists (with duplicates)
def invert_dictlist_dup(d):
    values = set(a for b in d.values() for a in b)
    reverse_d = dict((new_key, [key for key,value in d.items() if new_key in value]) for new_key in values)
    return reverse_d


In [2]:
# assign students in groups to k submissions.
def peer_assignment(groups,k,coverList=[],debug=False):
    """Given no cover, first generate a cover with the first few submissions"""
    """Then, generate the rest of the assignments"""
    
    submissions = groups.keys()
    
    # lookup for which submissions to exclude from a particular student.
    exclude = invert_dictlist(groups)
    students = exclude.keys()
    studentList = list(students);

 
    # load = ceil(number of students * k / number of submissions)
    # this is how many copies of random submission lists we need.
    load = int(math.ceil((len(students) * k) / len(submissions)))
    
    # Repeat the submissions so that they each occur load times.
    repeatedSubmissions = [x for x in submissions for i in range(load-1)]
    
    # initialize empty assignment and cover
    assignments = {s : [] for s in students}
    cover = {s : [] for s in students}
    
    # cover with the first few submissions. Every time a submission is used, remove it.
    # This way, each submission is repeated at most load # of times.
    for s in students:
        for currentSubmission in repeatedSubmissions:
            if currentSubmission not in exclude[s]:
                assignments[s].append(currentSubmission);
                repeatedSubmissions.remove(currentSubmission);
                cover[s].append(currentSubmission);
                break;
    
    assignments = peer_assignment_with_cover(groups,k,cover,coverList);
    
    if (assignments == -1) and debug:
        print("Couldn't generate good cover!");
    
    done = True;
    for s in students:
        if (len(assignments[s]) != k):
            done = False;
            
    if not(done) and debug:
        print("Not a good assignment!")
        return -1;
    
    return assignments;
    
def peer_assignment_with_cover_submissions(groups,k,coverSubmissions,coverList=[],debug=False):
    """Given a list of submissions, generate a cover using those submissions"""
    """Then, generate the rest of the assignments"""
    
    submissions = groups.keys()
    
    # lookup for which submissions to exclude from a particular student.
    exclude = invert_dictlist(groups)
    students = exclude.keys()
    studentList = list(students);

    # load = ceil(number of students * k / number of submissions)
    # this is how many copies of random submission lists we need.
    load = int(math.ceil((len(students) * k) / len(submissions)))
    
    repeatedCoverSubmissions = [x for x in coverSubmissions for i in range(load-1)]
    
    # initialize empty cover
    cover = {s : [] for s in students}
    
    # cover with the first few of coverSubmissions
    for s in students:
        for currentSubmission in repeatedCoverSubmissions:
            if currentSubmission not in exclude[s]:
                repeatedCoverSubmissions.remove(currentSubmission);
                cover[s].append(currentSubmission);
                break;
    
    for s in students:
        if len(assignments[s]) == 0:
            if (debug):
                print("Error: Submissions cannot cover students!")
            return -1;
        
    assignments = peer_assignment_with_cover(groups,k,cover,coverList);
    
    if (assignments == -1) and debug:
        print("Couldn't generate good cover!");
        return -1;
    
     
    done = True;
    for s in students:
        if (len(assignments[s]) != k):
            done = False;
            
    if not(done) and debug:
        print("Not a good assignment!")
        return -1;
    
    return assignments;
    
def peer_assignment_with_cover(groups,k,cover,coverList = [],debug=False):
    """Given an entire cover of (student,submission) pairs, generate the rest of the assignments"""
    
    
    for i in range(10000):
        
        submissions = groups.keys()
        submissionsList = list(submissions);
        
        # lookup for which submissions to exclude from a particular student.
        exclude = invert_dictlist(groups)
        students = exclude.keys()
        studentList = list(students);
    
     
        # load = ceil(number of students * k / number of submissions)
        # this is how many copies of random submission lists we need.
        load = int(math.ceil((len(students) * k) / len(submissions)))
    
        # Repeat the submissions so that they each occur load times.
        extras = len(students)*k - len(submissions)*(load - 1);
        repeatedSubmissions = [x for x in submissions for i in range(load-1)]
        more = submissionsList[:extras]
        repeatedSubmissions = repeatedSubmissions + more
        
    
        # Assign cover, and remove covered submissions from submissions list
        assignments = cover;
        for s in students:
            for currentSubmission in assignments[s]:
                repeatedSubmissions.remove(currentSubmission);
        
        permutedSubmissions = random.sample(repeatedSubmissions,len(repeatedSubmissions));
        repeatedStudents = [x for x in studentList for i in range(k-1)]
        permutedStudents = random.sample(repeatedStudents,len(repeatedStudents));
    
        for s in permutedStudents:
            for currentSubmission in permutedSubmissions:
                assignments[s].append(currentSubmission);
                if not(check_assignment(groups,assignments)):
                    assignments[s].remove(currentSubmission);
                else:
                    permutedSubmissions.remove(currentSubmission);
                    break;
        
        done = True;
        
        for s in students:
            if (len(assignments[s]) != k):
                done = False;
        
        if done:
            break;
    
    done = True;
    for s in students:
        if (len(assignments[s]) != k):
            done = False;
    
    if not(done) and debug:
        print("Bad Cover!")
        return -1;
      
        
    coverList.append(cover);
    return assignments
    

# make sure nobody is assigned own assignment or duplicates
def check_assignment(groups,assignments):
    # maps students to their own submission
    
    exclude = invert_dictlist(groups)

    passed = True
    
    for s in assignments.keys():
        if exclude[s] in assignments[s]:
            #print("Student " + s + " assigned to own submission\n")
            passed = False
        if len(assignments[s]) != len(set(assignments[s])):
            #print("Student " + s + " assigned duplicate submissions\n")
            passed = False
            
    return passed

In [4]:
# generate some group data
groups = { sub : [sub + x for x in ['1','2','3']] for sub in [ chr(ord('a') + z) for z in range(10)]}

# get assignments
coverList = [];
assignments = peer_assignment(groups,3,coverList);

# print assignments and cover
print(coverList)
invert_dictlist_dup(assignments)

[{'b2': ['i', 'c', 'a'], 'g2': ['i', 'd', 'f'], 'c1': ['i', 'a', 'd'], 'a2': ['h', 'f', 'b'], 'h3': ['j', 'a', 'f'], 'b3': ['a', 'i', 'd'], 'h2': ['i', 'a', 'j'], 'b1': ['i', 'f', 'd'], 'e1': ['h', 'a', 'b'], 'j3': ['e', 'g', 'b'], 'e3': ['h', 'c', 'd'], 'f1': ['i', 'e', 'c'], 'f2': ['e', 'c', 'd'], 'i2': ['e', 'f', 'a'], 'j2': ['e', 'g', 'i'], 'd2': ['e', 'c', 'h'], 'd3': ['e', 'g', 'b'], 'd1': ['i', 'j', 'g'], 'g3': ['h', 'a', 'b'], 'i3': ['h', 'c', 'g'], 'f3': ['h', 'g', 'd'], 'i1': ['e', 'j', 'g'], 'g1': ['h', 'c', 'b'], 'h1': ['j', 'c', 'd'], 'c3': ['e', 'a', 'b'], 'a3': ['h', 'f', 'j'], 'a1': ['j', 'f', 'b'], 'e2': ['j', 'g', 'f'], 'j1': ['c', 'g', 'd'], 'c2': ['j', 'b', 'f']}]


{'a': ['b2', 'c1', 'h3', 'b3', 'h2', 'e1', 'i2', 'g3', 'c3'],
 'b': ['a2', 'e1', 'j3', 'd3', 'g3', 'g1', 'c3', 'a1', 'c2'],
 'c': ['b2', 'e3', 'f1', 'f2', 'd2', 'i3', 'g1', 'h1', 'j1'],
 'd': ['g2', 'c1', 'b3', 'b1', 'e3', 'f2', 'f3', 'h1', 'j1'],
 'e': ['j3', 'f1', 'f2', 'i2', 'j2', 'd2', 'd3', 'i1', 'c3'],
 'f': ['g2', 'a2', 'h3', 'b1', 'i2', 'a3', 'a1', 'e2', 'c2'],
 'g': ['j3', 'j2', 'd3', 'd1', 'i3', 'f3', 'i1', 'e2', 'j1'],
 'h': ['a2', 'e1', 'e3', 'd2', 'g3', 'i3', 'f3', 'g1', 'a3'],
 'i': ['b2', 'g2', 'c1', 'b3', 'h2', 'b1', 'f1', 'j2', 'd1'],
 'j': ['h3', 'h2', 'd1', 'i1', 'h1', 'a3', 'a1', 'e2', 'c2']}

In [122]:
peer_assignment(groups,3,True)

{'a1': ['j', 'b', 'e'],
 'a2': ['b', 'i', 'e'],
 'a3': ['h', 'g', 'd'],
 'b1': ['h', 'a', 'i'],
 'b2': ['f', 'j', 'd'],
 'b3': ['j', 'a', 'g'],
 'c1': ['a', 'e', 'i'],
 'c2': ['j', 'i', 'f'],
 'c3': ['j', 'f', 'g'],
 'd1': ['a', 'f', 'g'],
 'd2': ['j', 'i', 'e'],
 'd3': ['g', 'h', 'c'],
 'e1': ['b', 'a', 'f'],
 'e2': ['h', 'i', 'd'],
 'e3': ['b', 'f', 'd'],
 'f1': ['b', 'e', 'g'],
 'f2': ['j', 'c', 'g'],
 'f3': ['h', 'c', 'j'],
 'g1': ['b', 'e', 'f'],
 'g2': ['j', 'i', 'c'],
 'g3': ['h', 'd', 'f'],
 'h1': ['b', 'e', 'i'],
 'h2': ['b', 'g', 'i'],
 'h3': ['a', 'e', 'd'],
 'i1': ['b', 'e', 'g'],
 'i2': ['a', 'c', 'd'],
 'i3': ['a', 'c', 'h'],
 'j1': ['a', 'f', 'c'],
 'j2': ['h', 'd', 'c'],
 'j3': ['h', 'd', 'c']}

In [5]:
# generate groups

L = [chr(ord('a') + z) for z in range(26)];
bigL = [x + y for x in L for y in L]
bigGroups = {sub : [sub + x for x in ['1','2','3']] for sub in bigL[:90]};
bigGroups['aa'].remove('aa1');
bigGroups['ab'].remove('ab1');
bigGroups['ac'].remove('ac1');
bigGroups['ad'].remove('ad1');
bigGroups['ae'].remove('ae1');
bigGroups['af'].remove('af1');
bigGroups['ag'].remove('ag1');
bigGroups['ah'].remove('ah1');
bigGroups['ai'].remove('ai1');
bigGroups['aj'].remove('aj1');
bigGroups['ak'].remove('ak1');
bigGroups['al'].remove('al1');
bigGroups['am'].remove('am1');
bigGroups['an'].remove('an1');
bigGroups['ao'].remove('ao1');
bigGroups['ap'].remove('ap1');
bigGroups['aq'].remove('aq1');
bigGroups['ar'].remove('ar1');
bigGroups['as'].remove('as1');
bigGroups['at'].remove('at1');
bigGroups['au'].remove('au1');
bigGroups['av'].remove('av1');
bigGroups['aw'].remove('aw1');
bigGroups['ax'].remove('ax1');
bigGroups['ay'].remove('ay1');
bigGroups['az'].remove('az1');


# generate assignments
coverList = [];
assignments = peer_assignment(bigGroups,3,coverList)

# assignments
print(coverList)
invert_dictlist_dup(assignments)

[{'an3': ['ce', 'cw', 'bn'], 'co3': ['ce', 'cg', 'ck'], 'cb2': ['ce', 'af', 'bn'], 'co2': ['ce', 'bd', 'dg'], 'bx1': ['ce', 'bn', 'bp'], 'ac3': ['ce', 'db', 'ai'], 'dh1': ['ce', 'bm', 'bf'], 'av2': ['ce', 'dg', 'cb'], 'bb1': ['at', 'bn', 'bs'], 'dk3': ['be', 'au', 'dc'], 'de1': ['be', 'ct', 'bu'], 'ay3': ['be', 'ba', 'cg'], 'ah2': ['cx', 'bu', 'ca'], 'cy1': ['be', 'bv', 'dc'], 'cw2': ['be', 'cu', 'av'], 'bq3': ['be', 'cj', 'br'], 'cs3': ['bw', 'ah', 'bh'], 'aw3': ['cy', 'cr', 'ck'], 'ct2': ['cy', 'cm', 'bv'], 'cv2': ['cy', 'cj', 'ct'], 'bj2': ['bk', 'cy', 'dj'], 'cc3': ['cy', 'bh', 'dg'], 'cj2': ['cy', 'af', 'cu'], 'ak3': ['dk', 'bi', 'bz'], 'bn1': ['cy', 'cr', 'ao'], 'cz1': ['cy', 'cn', 'cb'], 'bf1': ['al', 'av', 'bv'], 'be1': ['al', 'ca', 'bl'], 'bv1': ['al', 'bp', 'ao'], 'bh2': ['al', 'ay', 'di'], 'cj3': ['al', 'di', 'aq'], 'bd1': ['al', 'ae', 'az'], 'cr1': ['dk', 'cf', 'au'], 'cy2': ['ap', 'ah', 'cl'], 'ag3': ['ap', 'bh', 'bi'], 'bb3': ['ap', 'bc', 'cj'], 'dd2': ['ap', 'bz', 'dc'],

{'aa': ['bs1', 'au2', 'df1', 'bw3', 'bn3', 'dg1', 'dd3', 'bj3', 'bi2'],
 'ab': ['cf1', 'bs1', 'at2', 'cn2', 'dk2', 'bi1', 'ai2', 'cu1'],
 'ac': ['bs2', 'av3', 'ad3', 'cm1', 'ad2', 'br2', 'bl2', 'dd1'],
 'ad': ['bt1', 'dj3', 'df3', 'cs2', 'ct1', 'cr2', 'bd2', 'cq2'],
 'ae': ['bd1', 'bn3', 'bo3', 'af2', 'bc2', 'bc3', 'am2', 'bu1'],
 'af': ['cb2', 'cj2', 'cf3', 'cm1', 'bg3', 'bt2', 'ae2', 'al3'],
 'ag': ['cp2', 'bs1', 'bo1', 'cz2', 'ap2', 'cl2', 'ai2', 'cu3'],
 'ah': ['cs3', 'cy2', 'cq1', 'ay2', 'bs2', 'dg2', 'al3', 'an2'],
 'ai': ['ac3', 'bz1', 'bn3', 'ab2', 'ch2', 'bv3', 'am3', 'bw2'],
 'aj': ['cp2', 'dc2', 'cq3', 'ae2', 'bw1', 'ap2', 'bz3', 'ao3'],
 'ak': ['af3', 'bw2', 'df3', 'ar2', 'aq2', 'db2', 'ci3', 'bd2'],
 'al': ['bf1', 'be1', 'bv1', 'bh2', 'cj3', 'bd1', 'dg3', 'bl2', 'ca3'],
 'am': ['aq3', 'br3', 'dc3', 'cd3', 'ch3', 'cj1', 'db3', 'bq2', 'ao2'],
 'an': ['ci2', 'dh3', 'bh1', 'bg3', 'cv3', 'aq2', 'bx2', 'dh2'],
 'ao': ['bn1', 'bv1', 'au2', 'br3', 'ad2', 'af3', 'cr3', 'bo2'],
 'ap

In [119]:
peer_assignment(bigGroups,3)

9
90
244
12


{'aa2': ['cc', 'ai', 'cs'],
 'aa3': ['ao', 'ab', 'bk'],
 'ab2': ['bg', 'bu', 'ci'],
 'ab3': ['co', 'dh', 'bi'],
 'ac2': ['df', 'bl', 'cj'],
 'ac3': ['ao', 'de', 'au'],
 'ad2': ['bb', 'cb', 'cn'],
 'ad3': ['co', 'cb', 'by'],
 'ae2': ['cf', 'dj', 'cj'],
 'ae3': ['bf', 'cb', 'ay'],
 'af2': ['ad', 'bv', 'ak'],
 'af3': ['ad', 'aj', 'cn'],
 'ag2': ['ah', 'bp', 'bc'],
 'ag3': ['cc', 'ab', 'ac'],
 'ah2': ['dl', 'bn', 'dg'],
 'ah3': ['ao', 'ae', 'bs'],
 'ai2': ['bd', 'cx', 'cu'],
 'ai3': ['aw', 'ba', 'bl'],
 'aj2': ['dc', 'cp', 'cj'],
 'aj3': ['bv', 'cm', 'cr'],
 'ak2': ['df', 'dh', 'az'],
 'ak3': ['bv', 'cu', 'dd'],
 'al2': ['bv', 'az', 'ch'],
 'al3': ['ca', 'az', 'au'],
 'am2': ['bo', 'bh', 'de'],
 'am3': ['aw', 'bu', 'aa'],
 'an2': ['bo', 'bg', 'af'],
 'an3': ['bz', 'bm', 'cz'],
 'ao2': ['dl', 'al', 'ae'],
 'ao3': ['ad', 'cz', 'da'],
 'ap2': ['ax', 'ch', 'av'],
 'ap3': ['cf', 'ba', 'be'],
 'aq2': ['bd', 'bl', 'bt'],
 'aq3': ['cc', 'bu', 'ak'],
 'ar2': ['bo', 'by', 'bq'],
 'ar3': ['aq', 'cl',